## 7.1 不用Sequential模型的解决方案：Keras函数式API
### 7.1.1 函数式API简介

In [ ]:
from keras import Input, layers

# 一个张量
input_tensor = Input(shape=(32,))

# 一个层是一个函数
dense = layers.Dense(32, activation='relu')

# 可以在一个张量上调用一个层，它会返回一个张量
output_tensor = dense(input_tensor)

In [ ]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 对应的函数式API实现
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# Model类将输入张量和输出张量转换为一个模型
model = Model(input_tensor, output_tensor)

model.summary()

In [ ]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input, output_tensor)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
# 生成用于训练的虚构Numpy数据
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)
print(score)

### 7.1.2 多输入模型
**代码清单 7-1** 用函数式API实现双输入问答模型

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 文本输入是一个长度可变的整数序列。注意，你可以选择对输入进行命名
text_input = Input(shape=(100,), dtype='int32', name='text')

# 将输入嵌入长度为64的向量
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)

# 利用LSTM将向量编码为单个向量
encoded_text = layers.LSTM(32)(embedded_text)

# 对问题进行相同的处理（使用不同的层实例）
question_input = Input(shape=(100,),
                       dtype='int32',
                       name='question')

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 将编码后的问题和文本连接起来
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# 在上面添加一个softmax分类器
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# 在模型实例化时，指定两个输入和输出
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

**代码清单 7-2** 将数据输入到多输入模型中

In [ ]:
import numpy as np
import keras

num_samples = 1000
max_length = 100

# 生成虚构的Numpy数据
text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))

answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
# 回答是one-hot编码的，不是整数
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

# 使用输入组成的列表来拟合
model.fit([text, question], answers, epochs=10, batch_size=128)

# 使用输入组成的字典来拟合（只有对输入进行命名之后才能用这种方法）
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

### 7.1.3 多输出模型
**代码清单 7-3** 用函数式API实现一个三输出模型

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 注意，输出层都具有名称
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

**代码清单7-4** 多输出模型的编译选项：多重损失

In [ ]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# 与上述写法等效（只有输出层具有名称时才能用这种写法）
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

**代码清单 7-5** 多输出模型的编译选项：损失加权

In [ ]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

# 与上述写法等效（只有输出层具有名称时才能用这种写法）
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

**代码清单 7-6** 将数据输入到多输出模型中

In [ ]:
'''
# 假设age_targets、income_targets和gender_targets都是Numpy数组
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# 与上述写法等效（只有输出层具有名称时才能用这种写法）
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)
'''

### 7.1.4 层组成的有向无环图
**1. Inception 模块**

In [ ]:
'''
from keras import layers

# 每个分支都具有相同的步幅值（2），这对于保持所有分支输出具有
# 相同的尺寸是很有必要的，这样你才能将它们连接在一起
branch_a = layers.Conv2D(128, 1,
                         activation='relu', strides=2)(x)
# 在这个分支中，空间卷积层用到了步幅
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3,
                         activation='relu', strides=2)(branch_b)
# 在这个分支中，平均池化层用到了步幅
branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3,
                         activation='relu', strides=2)(branch_d)
# 将分支输出连接在一起，得到模块输出
output = layers.concatenate(
    [branch_a, branch_b, branch_c, branch_d], axis=-1)
'''                       

**2. 残差连接**

In [ ]:
'''
from keras import layers

x = ...
# 对x进行变换
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])  # 将原始x与输出特征相加
'''

In [ ]:
'''
from keras import layers

x = ...
# 对x进行变换
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# 将使用1x1卷积，将原始x张量线性下采样为与y具有相同的形状
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])  # 将原始x与输出特征相加
'''

**7.1.5 共享层权重**

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

# 将一个LSTM层实例化一次
lstm = layers.LSTM(32)

# 构建模型的左分支：输入是长度128的向量组成的变长序列
left_input = Input(shape=(50, 128))
left_output = lstm(left_input)

# 构建模型的右分支：如果调用已有的层实例，那么就会重复使用它的权重
right_input = Input(shape=(50, 128))
right_output = lstm(right_input)

# 在上面构建一个分类器
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# 将模型实例化并训练：训练这种模型时，基于两个输入对LSTM层的权重进行更新
model = Model([left_input, right_input], predictions)
# model.fit([left_data, right_data], targets)

### 7.1.6 将模型作为层

In [ ]:
from keras import layers
from keras import applications
from keras import Input

# 图像处理基础模型是Xception网络（只包含卷积基）
xception_base = applications.Xception(weights=None,
                                      include_top=False)

# 输入是250x250的RGB图像
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 对相同的视觉模型调用两次
left_features = xception_base(left_input)
right_features = xception_base(right_input)

# 合并后的特征包含来自左右两个视觉输入中的信息
merged_features = layers.concatenate(
    [left_features, right_features], axis=-1)